# Warning
This notebook is under development- please use to evaluate entropy algorithms, 

# Experiments on entropy implementation towards speed


## Import libraries

In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import collections
import math
from scipy.stats import entropy
from math import log, e


from resurfemg.tmsisdk_lite import Poly5Reader
from resurfemg.config import Config

config = Config()

# Set a collection place for experiments

In [ ]:
# pull local helper functions
sys.path.insert(0, '../resurfemg')
import helper_functions as hf

In [ ]:
# do not rerun this cell
big_data_list = []

Below change the path to the root directory where you are keeping your EMGs and ventilator "Draeger" files

# Reruns should be done from this cell as the start

In [ ]:
root_emg_directory = config.get_directory('root_emg_directory')

In [ ]:
emg_pattern = os.path.join(root_emg_directory, '**/*.Poly5')
emg_and_draeger_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
draeger_files = []

for file in emg_and_draeger_files:
    if 'Draeger' in file:
        draeger_files.append(file)
    else:
        emg_files.append(file)

Now you can pick a file from the list, which have been numbered.

In [ ]:
list_of_numbers_strung = []
for i in range(len(emg_files)):
    list_of_numbers_strung.append(str(i))


btn = widgets.Dropdown(
    options=list_of_numbers_strung,
    value='0',
    description='Picked File:',
    disabled=False,
)
display(btn)

In [ ]:
number_chosen = int(btn.value)
file_chosen = emg_files[number_chosen] 
print("The file you chose is:", file_chosen)

In [ ]:
data_emg = Poly5Reader(file_chosen)
data_samples = data_emg.samples
emg_sample_rate = data_emg.sample_rate
converted_to_seconds =  []
converted_to_samples = []
for i in range(len(data_samples[0])):
    converted_to_seconds.append(i/emg_sample_rate)
    converted_to_samples.append(i)

In [ ]:
%matplotlib inline
# set up plotn
x = data_samples
fig, axis = plt.subplots(nrows = 3, ncols = 2, figsize=(6, 6))
#ax.set_ylim([-4, 4])
axis[0,0].grid(True)
axis[0,0].plot(x[0])
axis[0,0].set(title='leads in samples')
axis[1,0].plot(x[1])
axis[2,0].plot(x[2])
axis[0,1].set(title='leads in seconds')
axis[0,1].grid(True)
axis[0,1].plot(converted_to_seconds,x[0])
axis[1,1].plot(converted_to_seconds,x[1])
axis[2,1].plot(converted_to_seconds,x[2])

That is the whole unfiltered EMG, but you probably want to examine a part. You will also want to examine something filtered down to only the EMG components. Therefore we will filter off only the EMG components with an ICA in addtion to the filter we will play with.

Now you can filter down to which part you want to see. 

Do you want to cut and see the file in samples or seconds?

In [ ]:
y_axis = widgets.Dropdown(
    options=["Samples","Seconds"],
    value='Samples',
    description="Select View Option",
    disabled=False,
)
display(y_axis)


In [ ]:
time_view= y_axis.value
time_view

In [ ]:
# type in start number and press return for it to update
start = input()


In [ ]:
# type in end number and press return for it to update
end= input()


Do we want to?
# Add warning cell if end is less than start, or numbers are out of range?

In [ ]:
start_s = float(start)* emg_sample_rate
end_s = float(end)*emg_sample_rate

In [ ]:
if time_view == 'Samples':
    # nox examine at a certain scale- from point a to b as samples
    x = data_samples
    fig, (ax_1,ax_2,ax_3) = plt.subplots(nrows = 3, figsize=(6, 4))
    ax_1.grid(True)
    ax_1.plot(x[0][int(start):int(end)])
    ax_1.set(title='leads, samples')
    ax_2.plot(x[1][int(start):int(end)])
    ax_3.plot(x[2][int(start):int(end)])
    
if time_view == 'Seconds':
    # nox examine at a certain scale- from point a to b as samples
    x_for_secs = data_samples
    
    converter_for_sample_number =90
    
    y = converted_to_seconds
    fig, (ax_1,ax_2,ax_3) = plt.subplots(nrows = 3, figsize=(6, 4))
    ax_1.grid(True)
    ax_1.plot(y[int(start_s):int(end_s)],x[0][int(start_s):int(end_s)])
    ax_1.set(title='leads, seconds')
    ax_2.plot(y[int(start_s):int(end_s)],x[1][int(start_s):int(end_s)])
    ax_3.plot(y[int(start_s):int(end_s)],x[2][int(start_s):int(end_s)])
    

    

Happy with your selection? If not redo, then we can see how the filter the selection in a basic pipleline before extracint entropy.

In [ ]:
def working_pipeline_pre_entropy(our_chosen_samples): 
    cut_file_data = hf.bad_end_cutter_for_samples(our_chosen_samples, percent_to_cut=3, tolerance_percent=5)
    bd_filtered_file_data = hf.emg_bandpass_butter_sample(cut_file_data, 5, 450, 2048, output='sos')
    # step 3 end-cutting again to get rid of filtering artifacts
    re_cut_file_data = hf.bad_end_cutter_for_samples(bd_filtered_file_data, percent_to_cut=3, tolerance_percent=5)
    # skip step4 and do step 5 ICA
    components = hf.compute_ICA_two_comp(re_cut_file_data)
    #     the picking step!
    emg= hf.pick_more_peaks_array(components)
    # now process it in final steps
    abs_values = abs(emg)
    final_envelope_d = hf.emg_highpass_butter(abs_values, 150, 2048)
    
        
    return final_envelope_d

In [ ]:
processed_data_emg = working_pipeline_pre_entropy(data_samples)
plt.plot(processed_data_emg)

In [ ]:

def entropy_scipy(sli, base=None):
    """
    This function wraps scipy.stats entropy for use in the resurfemg library, it can be
    used in a slice iterator as a drop-in substitute for the hf.entropical
    except it is a true entropy.
    
    returns:
    """
    
    value,counts = np.unique(sli, return_counts=True)
    entropy_count = entropy(counts, base=base)
    return entropy_count
    


In [ ]:


def sampen(data, emb_dim=2, tolerance=None, dist=rowwise_chebyshev,
           closed=False, debug_plot=False, debug_data=False, plot_file=None):
  """
  The following code is adapted from openly licensed code written by 
  Christopher Schölzel in his package nolds (NOnLinear measures for Dynamical Systems). 
  It computes the sample entropy of time sequence data.
  
  Reference:
    .. [se_1] J. S. Richman and J. R. Moorman, “Physiological time-series
       analysis using approximate entropy and sample entropy,”
       American Journal of Physiology-Heart and Circulatory Physiology,
       vol. 278, no. 6, pp. H2039–H2049, 2000.

  Args:
    data (array-like of float):
      input data
  Kwargs:
    emb_dim (int):
      the embedding dimension (length of vectors to compare)
    tolerance (float):
      distance threshold for two template vectors to be considered equal
      (default: 0.2 * std(data) at emb_dim = 2, corrected for dimension effect
      for other values of emb_dim)
    dist (function (2d-array, 1d-array) -> 1d-array):
      distance function used to calculate the distance between template
      vectors. Sampen is defined using ``rowwise_chebyshev``. You should only
      use something else, if you are sure that you need it.
    closed (boolean):
      if True, will check for vector pairs whose distance is in the closed
      interval [0, r] (less or equal to r), otherwise the open interval
      [0, r) (less than r) will be used
    debug_plot (boolean):
      if True, a histogram of the individual distances for m and m+1
    debug_data (boolean):
      if True, debugging data will be returned alongside the result
    plot_file (str):
      if debug_plot is True and plot_file is not None, the plot will be saved
      under the given file name instead of directly showing it through
      ``plt.show()``
  Returns:
    float:
      the sample entropy of the data (negative logarithm of ratio between
      similar template vectors of length emb_dim + 1 and emb_dim)
    [c_m, c_m1]:
      list of two floats: count of similar template vectors of length emb_dim
      (c_m) and of length emb_dim + 1 (c_m1)
    [float list, float list]:
      Lists of lists of the form ``[dists_m, dists_m1]`` containing the
      distances between template vectors for m (dists_m)
      and for m + 1 (dists_m1).
  """
  data = np.asarray(data)

  if tolerance is None:

    tolerance = np.std(data, ddof=1) * 0.1164 * (0.5627 * np.log(emb_dim) + 1.3334)
  n = len(data)

  # build matrix of "template vectors"
  # (all consecutive subsequences of length m)
  # x0 x1 x2 x3 ... xm-1
  # x1 x2 x3 x4 ... xm
  # x2 x3 x4 x5 ... xm+1
  # ...
  # x_n-m-1     ... xn-1

  # since we need two of these matrices for m = emb_dim and m = emb_dim +1,
  # we build one that is large enough => shape (emb_dim+1, n-emb_dim)

  # note that we ignore the last possible template vector with length emb_dim,
  # because this vector has no corresponding vector of length m+1 and thus does
  # not count towards the conditional probability
  # (otherwise first dimension would be n-emb_dim+1 and not n-emb_dim)
  tVecs = delay_embedding(np.asarray(data), emb_dim+1, lag=1)
  plot_data = []
  counts = []
  for m in [emb_dim, emb_dim + 1]:
    counts.append(0)
    plot_data.append([])
    # get the matrix that we need for the current m
    tVecsM = tVecs[:n - m + 1, :m]
    # successively calculate distances between each pair of template vectors
    for i in range(len(tVecsM) - 1):
      dsts = dist(tVecsM[i + 1:], tVecsM[i])
      if debug_plot or debug_data:
        plot_data[-1].extend(dsts)
      # count how many distances are smaller than the tolerance
      if closed:
        counts[-1] += np.sum(dsts <= tolerance)
      else:
        counts[-1] += np.sum(dsts < tolerance)
  if counts[0] > 0 and counts[1] > 0:
    saen = -np.log(1.0 * counts[1] / counts[0])
  else:
    # log would be infinite or undefined => cannot determine saen
    zcounts = []
    if counts[0] == 0:
      zcounts.append("emb_dim")
    if counts[1] == 0:
      zcounts.append("emb_dim + 1")
    warnings.warn(
      (
        "Zero vectors are within tolerance for %s. " \
        + "Consider raising the tolerance parameter to avoid %s result."
      ) % (" and ".join(zcounts), "NaN" if len(zcounts) == 2 else "inf"),
      RuntimeWarning
    )
    if counts[0] == 0 and counts[1] == 0:
      saen = np.nan
    elif counts[0] == 0:
      saen = -np.inf
    else:
      saen = np.inf
  if debug_plot:
    plot_dists(plot_data, tolerance, m, title="sampEn = {:.3f}".format(saen),
               fname=plot_file)
  if debug_data:
    return (saen, counts, plot_data)
  else:
    return saen


In [ ]:


def sampen2(data, emb_dim=2, tolerance=None, dist=rowwise_chebyshev,
           closed=False ):#, debug_plot=False, debug_data=False, plot_file=None):
  """
  The following code is adapted from openly licensed code written by 
  Christopher Schölzel in his package nolds (NOnLinear measures for Dynamical Systems). 
  It computes the sample entropy of time sequence data.
  
  Reference:
    .. [se_1] J. S. Richman and J. R. Moorman, “Physiological time-series
       analysis using approximate entropy and sample entropy,”
       American Journal of Physiology-Heart and Circulatory Physiology,
       vol. 278, no. 6, pp. H2039–H2049, 2000.

  Args:
    data (array-like of float):
      input data
  Kwargs:
    emb_dim (int):
      the embedding dimension (length of vectors to compare)
    tolerance (float):
      distance threshold for two template vectors to be considered equal
      (default: 0.2 * std(data) at emb_dim = 2, corrected for dimension effect
      for other values of emb_dim)
    dist (function (2d-array, 1d-array) -> 1d-array):
      distance function used to calculate the distance between template
      vectors. Sampen is defined using ``rowwise_chebyshev``. You should only
      use something else, if you are sure that you need it.
    closed (boolean):
      if True, will check for vector pairs whose distance is in the closed
      interval [0, r] (less or equal to r), otherwise the open interval
      [0, r) (less than r) will be used
    debug_plot (boolean):
      if True, a histogram of the individual distances for m and m+1
    debug_data (boolean):
      if True, debugging data will be returned alongside the result
    plot_file (str):
      if debug_plot is True and plot_file is not None, the plot will be saved
      under the given file name instead of directly showing it through
      ``plt.show()``
  Returns:
    float:
      the sample entropy of the data (negative logarithm of ratio between
      similar template vectors of length emb_dim + 1 and emb_dim)
    [c_m, c_m1]:
      list of two floats: count of similar template vectors of length emb_dim
      (c_m) and of length emb_dim + 1 (c_m1)
    [float list, float list]:
      Lists of lists of the form ``[dists_m, dists_m1]`` containing the
      distances between template vectors for m (dists_m)
      and for m + 1 (dists_m1).
  """
  data = np.asarray(data)

  if tolerance is None:

    tolerance = np.std(data, ddof=1) * 0.1164 * (0.5627 * np.log(emb_dim) + 1.3334)
  n = len(data)

  # build matrix of "template vectors"
  # (all consecutive subsequences of length m)
  # x0 x1 x2 x3 ... xm-1
  # x1 x2 x3 x4 ... xm
  # x2 x3 x4 x5 ... xm+1
  # ...
  # x_n-m-1     ... xn-1

  # since we need two of these matrices for m = emb_dim and m = emb_dim +1,
  # we build one that is large enough => shape (emb_dim+1, n-emb_dim)

  # note that we ignore the last possible template vector with length emb_dim,
  # because this vector has no corresponding vector of length m+1 and thus does
  # not count towards the conditional probability
  # (otherwise first dimension would be n-emb_dim+1 and not n-emb_dim)
  tVecs = delay_embedding(np.asarray(data), emb_dim+1, lag=1)
  plot_data = []
  counts = []
  for m in [emb_dim, emb_dim + 1]:
    counts.append(0)
    plot_data.append([])
    # get the matrix that we need for the current m
    tVecsM = tVecs[:n - m + 1, :m]
    # successively calculate distances between each pair of template vectors
    for i in range(len(tVecsM) - 1):
      dsts = dist(tVecsM[i + 1:], tVecsM[i])
#       if debug_plot or debug_data:
#         plot_data[-1].extend(dsts)
      # count how many distances are smaller than the tolerance
      if closed:
        counts[-1] += np.sum(dsts <= tolerance)
      else:
        counts[-1] += np.sum(dsts < tolerance)
  if counts[0] > 0 and counts[1] > 0:
    saen = -np.log(1.0 * counts[1] / counts[0])
  else:
    # log would be infinite or undefined => cannot determine saen
    zcounts = []
    if counts[0] == 0:
      zcounts.append("emb_dim")
    if counts[1] == 0:
      zcounts.append("emb_dim + 1")
    warnings.warn(
      (
        "Zero vectors are within tolerance for %s. " \
        + "Consider raising the tolerance parameter to avoid %s result."
      ) % (" and ".join(zcounts), "NaN" if len(zcounts) == 2 else "inf"),
      RuntimeWarning
    )
    if counts[0] == 0 and counts[1] == 0:
      saen = np.nan
    elif counts[0] == 0:
      saen = -np.inf
    else:
      saen = np.inf
#   if debug_plot:
#     plot_dists(plot_data, tolerance, m, title="sampEn = {:.3f}".format(saen),
#                fname=plot_file)
#   if debug_data:
#     return (saen, counts, plot_data)
#   else:
    return saen


In [ ]:
def rowwise_chebyshev(x, y):
  return np.max(np.abs(x - y), axis=1)

def delay_embedding(data, emb_dim, lag=1):
  """
  The following code is adapted from openly licensed code written by 
  Christopher Schölzel in his package nolds (NOnLinear measures for Dynamical Systems). 
  It performs a time-delay embedding of a time series
  
  :param data: array-like
  :type data: array
  :param emb_dim: the embedded dimension
  :type emb_dim: int
  :param lag: the lag between elements in the embedded vectors
  :type lag: int
  
  :returns: matrix_vectors
  :rtype: ~nd.array
  """
  data = np.asarray(data)
  min_len = (emb_dim - 1) * lag + 1
  if len(data) < min_len:
    msg = "cannot embed data of length {} with embedding dimension {} " \
        + "and lag {}, minimum required length is {}"
    raise ValueError(msg.format(len(data), emb_dim, lag, min_len))
  m = len(data) - min_len + 1
  indices = np.repeat([np.arange(emb_dim) * lag], m, axis=0)
  indices += np.arange(m).reshape((m, 1))
  matrix_vectors = data[indices]
  return matrix_vectors



def sampen(data, emb_dim=2, tolerance=None, dist=rowwise_chebyshev,
           closed=False, debug_plot=False, debug_data=False, plot_file=None):
  """
  The following code is adapted from openly licensed code written by 
  Christopher Schölzel in his package nolds (NOnLinear measures for Dynamical Systems). 
  It computes the sample entropy of time sequence data.
  
  Reference:
    .. [se_1] J. S. Richman and J. R. Moorman, “Physiological time-series
       analysis using approximate entropy and sample entropy,”
       American Journal of Physiology-Heart and Circulatory Physiology,
       vol. 278, no. 6, pp. H2039–H2049, 2000.
    
  Args:
    data (array-like of float):
      input data
  Kwargs:
    emb_dim (int):
      the embedding dimension (length of vectors to compare)
    tolerance (float):
      distance threshold for two template vectors to be considered equal
      (default: 0.2 * std(data) at emb_dim = 2, corrected for dimension effect
      for other values of emb_dim)
    dist (function (2d-array, 1d-array) -> 1d-array):
      distance function used to calculate the distance between template
      vectors. Sampen is defined using ``rowwise_chebyshev``. You should only
      use something else, if you are sure that you need it.
    closed (boolean):
      if True, will check for vector pairs whose distance is in the closed
      interval [0, r] (less or equal to r), otherwise the open interval
      [0, r) (less than r) will be used
    debug_plot (boolean):
      if True, a histogram of the individual distances for m and m+1
    debug_data (boolean):
      if True, debugging data will be returned alongside the result
    plot_file (str):
      if debug_plot is True and plot_file is not None, the plot will be saved
      under the given file name instead of directly showing it through
      ``plt.show()``
  Returns:
    float:
      the sample entropy of the data (negative logarithm of ratio between
      similar template vectors of length emb_dim + 1 and emb_dim)
    [c_m, c_m1]:
      list of two floats: count of similar template vectors of length emb_dim
      (c_m) and of length emb_dim + 1 (c_m1)
    [float list, float list]:
      Lists of lists of the form ``[dists_m, dists_m1]`` containing the
      distances between template vectors for m (dists_m)
      and for m + 1 (dists_m1).
  """
  data = np.asarray(data)

  if tolerance is None:

    tolerance = np.std(data, ddof=1) * 0.1164 * (0.5627 * np.log(emb_dim) + 1.3334)
  n = len(data)

  # build matrix of "template vectors"
  # (all consecutive subsequences of length m)
  # x0 x1 x2 x3 ... xm-1
  # x1 x2 x3 x4 ... xm
  # x2 x3 x4 x5 ... xm+1
  # ...
  # x_n-m-1     ... xn-1

  # since we need two of these matrices for m = emb_dim and m = emb_dim +1,
  # we build one that is large enough => shape (emb_dim+1, n-emb_dim)

  # note that we ignore the last possible template vector with length emb_dim,
  # because this vector has no corresponding vector of length m+1 and thus does
  # not count towards the conditional probability
  # (otherwise first dimension would be n-emb_dim+1 and not n-emb_dim)
  tVecs = delay_embedding(np.asarray(data), emb_dim+1, lag=1)
  plot_data = []
  counts = []
  for m in [emb_dim, emb_dim + 1]:
    counts.append(0)
    plot_data.append([])
    # get the matrix that we need for the current m
    tVecsM = tVecs[:n - m + 1, :m]
    # successively calculate distances between each pair of template vectors
    for i in range(len(tVecsM) - 1):
      dsts = dist(tVecsM[i + 1:], tVecsM[i])
      if debug_plot or debug_data:
        plot_data[-1].extend(dsts)
      # count how many distances are smaller than the tolerance
      if closed:
        counts[-1] += np.sum(dsts <= tolerance)
      else:
        counts[-1] += np.sum(dsts < tolerance)
  if counts[0] > 0 and counts[1] > 0:
    saen = -np.log(1.0 * counts[1] / counts[0])
  else:
    # log would be infinite or undefined => cannot determine saen
    zcounts = []
    if counts[0] == 0:
      zcounts.append("emb_dim")
    if counts[1] == 0:
      zcounts.append("emb_dim + 1")
    warnings.warn(
      (
        "Zero vectors are within tolerance for %s. " \
        + "Consider raising the tolerance parameter to avoid %s result."
      ) % (" and ".join(zcounts), "NaN" if len(zcounts) == 2 else "inf"),
      RuntimeWarning
    )
    if counts[0] == 0 and counts[1] == 0:
      saen = np.nan
    elif counts[0] == 0:
      saen = -np.inf
    else:
      saen = np.inf
  if debug_plot:
    plot_dists(plot_data, tolerance, m, title="sampEn = {:.3f}".format(saen),
               fname=plot_file)
  if debug_data:
    return (saen, counts, plot_data)
  else:
    return saen


def entropy_maker(array, method='nolds', base=None, emb_dim=2, tolerance=None, dist=rowwise_chebyshev,
           closed=False, debug_plot=False, debug_data=False, plot_file=None):
    """
    The following code allows a user to input an array and calculate either a time-series specific
    entropy i.e. the nolds or a more general Shannon entropy as calculated in scipy. It calls 
    entropy functions in the file.
    
    """
    if method == 'scipy':
        output = entropy_scipy(array, base=base)
    elif method == 'nolds':
        output = sampen(array)
    else:
        print('your method is not an option, we have defaulted to noal')
        output = sampen(array)
    return output

In [ ]:
array_here = np.array([1,2,3,4,5,6,7,8,9,9,1,2,3,4,5,6,1,1,1,1,1,1,1,1,2,2,2,2,2,7,2])

In [ ]:
entropy_scipy(array_here, base=None)

In [ ]:
hf.entropy_scipy(array_here, base=None)

In [ ]:
entropy_maker(
        array_here,
        method='nolds', base=None, 
        
)

In [ ]:
import warnings

In [ ]:
hf.entropy_maker(
        array_here,
        method='nolds', base=None, 
        
)

In [ ]:
# Now we want to select where the FIRST entropy cut_off is, in terms of maximum entropy
int_slider1 = widgets.IntSlider(
    min=1, max=100, step=1,
    description='percentage of maximum'
)
widgets.VBox(
    [

        int_slider1,


    ]
)

In [ ]:
# Now we want to select where the cut_off is

entropy_cutoff1 = widgets.Dropdown(
    options=["Mean","Half_range"],
    value='Mean',
    description="Select Entropy Cut off",
    disabled=False,
)
display(entropy_cutoff1)


In [ ]:
high_decision = (int_slider1.value)/100
# high_decision

### Try with new entropy

In [ ]:
def rounded_for_ent(stralist):
    rounded= np.round_(stralist, decimals = 5)
    return rounded
if time_view == 'Samples':
    big_list = rounded_for_ent(processed_data_emg[int(start):int(end)])# replace with whole array of time series!
else:
    big_list = rounded_for_ent(processed_data_emg[int(start_s):int(end_s)])
slice_length = 100
def sliceIterator(lst, sliceLen):
    for i in range(len(lst) - sliceLen + 1):
        yield lst[i:i + sliceLen]
index_hold = []
for slice in sliceIterator(big_list, slice_length):
    entropy_index = hf.entropy_scipy(slice)
    index_hold.append(entropy_index)

if entropy_cutoff1.value == 'Half_range':
    ndecision_cutoff = (np.max(index_hold) + np.min(index_hold))/2
else:# entropy_cutoff.value == 'Mean':
    ndecision_cutoff = np.mean(index_hold)



nrms_rolled = hf.vect_naive_rolling_rms(index_hold,100) # so rms is rms entropy

nhigh_decision_cutoff = (nrms_rolled.max() - nrms_rolled.min())* high_decision +nrms_rolled.min()
if time_view == 'Samples':
    plt.plot(converted_to_samples[int(start):(int(start) + len(nrms_rolled))],processed_data_emg[int(start):(int(start) + len(nrms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_samples[int(start):(int(start) + len(nrms_rolled))],nrms_rolled)
    plt.plot(converted_to_samples[int(start):(int(start) + len(nrms_rolled))],hf.zero_one_for_jumps_base(nrms_rolled,ndecision_cutoff))
    plt.plot(converted_to_samples[int(start):(int(start) + len(nrms_rolled))],hf.zero_one_for_jumps_base(nrms_rolled,nhigh_decision_cutoff), color= 'purple')
    plt.axhline(y = ndecision_cutoff, color = 'r', linestyle = '-')
    plt.axhline(y = nhigh_decision_cutoff, color = 'purple', linestyle = '-')
    
else:
    y = converted_to_seconds
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(nrms_rolled)))], processed_data_emg[int(start_s):(int(start_s)+len(nrms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(nrms_rolled)))],nrms_rolled)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(nrms_rolled)))],hf.zero_one_for_jumps_base(nrms_rolled,decision_cutoff))
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(nrms_rolled)))],hf.zero_one_for_jumps_base(nrms_rolled,high_decision_cutoff), color = 'purple')
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    plt.axhline(y = high_decision_cutoff, color = 'purple', linestyle = '-')
    

## Compare to broken entropy

In [ ]:
def rounded_for_ent(stralist):
    rounded= np.round_(stralist, decimals = 5)
    return rounded
if time_view == 'Samples':
    big_list = rounded_for_ent(processed_data_emg[int(start):int(end)])# replace with whole array of time series!
else:
    big_list = rounded_for_ent(processed_data_emg[int(start_s):int(end_s)])
slice_length = 100
def sliceIterator(lst, sliceLen):
    for i in range(len(lst) - sliceLen + 1):
        yield lst[i:i + sliceLen]
index_hold = []
for slice in sliceIterator(big_list, slice_length):
    entropy_index = hf.entropical(slice)
    index_hold.append(entropy_index)

if entropy_cutoff1.value == 'Half_range':
    decision_cutoff = (np.max(index_hold) + np.min(index_hold))/2
else:# entropy_cutoff.value == 'Mean':
    decision_cutoff = np.mean(index_hold)



rms_rolled = hf.vect_naive_rolling_rms(index_hold,100) # so rms is rms entropy

high_decision_cutoff = (rms_rolled.max() - rms_rolled.min())* high_decision +rms_rolled.min()
if time_view == 'Samples':
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],processed_data_emg[int(start):(int(start) + len(rms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],rms_rolled)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff))
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff), color= 'purple')
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    plt.axhline(y = high_decision_cutoff, color = 'purple', linestyle = '-')
    
else:
    y = converted_to_seconds
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))], processed_data_emg[int(start_s):(int(start_s)+len(rms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],rms_rolled)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff))
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff), color = 'purple')
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    plt.axhline(y = high_decision_cutoff, color = 'purple', linestyle = '-')
    

## Compare directly

In [ ]:
plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff), color = 'green', alpha=0.5)
plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(nrms_rolled,nhigh_decision_cutoff), color = 'red', alpha=0.5)


In [ ]:

plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],(np.array(hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff)) - np.array(hf.zero_one_for_jumps_base(nrms_rolled,nhigh_decision_cutoff))), color = 'red', alpha=0.5)
